In [1]:
from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis
from delierium.helpers import latexer
from sage.misc.html import html

In [2]:
x = var('x')
y=function('y')

In [3]:
ode=4*diff(y(x),x,x)*y(x) - 3* (diff(y(x),x)**2)-12*(y(x)**3)

In [4]:
p=prolongationODE(ode, y, x)

In [5]:
for _p in p:
    show(latexer(_p))

<p>\(-4 y \xi_{yy} y_{x}^3+4 y \phi_{yy} y_{x}^2-8 y \xi_{yx} y_{x}^2+6 \xi_{y} y_{x}^3-12 y \xi_{y} y_{x} y_{xx}-36 \phi y+8 y \phi_{yx} y_{x}-4 y \xi_{xx} y_{x}-6 \phi_{y} y_{x}^2+6 \xi_{x} y_{x}^2+4 y \phi_{y} y_{xx}-8 y \xi_{x} y_{xx}+4 y \phi_{xx}-6 \phi_{x} y_{x}+4 \phi y_{xx}\)</p>

In [ ]:
from IPython.core.debugger import set_trace
function('xi phi')
def infinitesimalsODE (ode, dependent, independent):
    prolongation = prolongationODE(ode, dependent, independent)[0].expand()
    s1  = solve(ode==0, diff(dependent(independent),independent,independent))
    ode1= prolongation.subs({s1[0].lhs() : s1[0].rhs()}).simplify()
    # so far it is checked manually and with mathematica
    l = [_ [0] for _ in ode1.coefficients(diff(dependent(independent), independent))]
    # remove hardcoded xi, phi
    janet = Janet_Basis(l, [xi, phi], [x, y])
    return janet

In [ ]:
ode=4*diff(y(x),x,x)*y(x) - 3* diff(y(x),x)**2-12*y(x)**3
inf = infinitesimalsODE(ode, y ,x)